# Training a Multitask GP model for Computer Model Emulation

Based on this example: https://docs.gpytorch.ai/en/v1.2.0/examples/03_Multitask_Exact_GPs/Multitask_GP_Regression.html

### Import necessary libraries 

In [42]:
%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import r2_score

import math
import torch
import gpytorch
import botorch

In [43]:
from calculations_load import *
from configurations import *

In [44]:
#prepare the directories for model design and outputs
#run this cell only once, or else it will hang!
#! sh prepare.sh

### Loading Computer Model inputs and outputs

In [45]:
system_str = 'Pb-Pb-2760'
design_file = 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_main_PbPb-2760.dat'
design = pd.read_csv(design_file)
design = design.drop("idx", axis=1)

#delete bad design points
drop_indices = list(delete_design_pts_set)
design = design.drop(drop_indices)

#choose features (inputs)
#feature_cols = ['norm', 'trento_p'] #specific choices
feature_cols = design.keys().values #all of them 

X = design[feature_cols]

#perform a transformation of the design, swapping the original parameters
#for inputs more naturally/simply related to the outputs
do_transform_design = True
if do_transform_design:
    X = transform_design(X.values)
    
n_features = X.shape[1]

n_design = SystemsInfo["Pb-Pb-2760"]["n_design"]
npt = n_design - len(delete_design_pts_set)

Y = np.array([])
Y_std = np.array([])
for pt in range(npt):
    for obs in active_obs_list['Pb-Pb-2760']:
        Y = np.append( Y, trimmed_model_data[system_str][pt, idf][obs]['mean'][:], axis=0)
        Y_std = np.append( Y_std, trimmed_model_data[system_str][pt, idf][obs]['err'][:], axis=0)
        
Y = Y.reshape(X.shape[0], -1)
Y_std = Y_std.reshape(X.shape[0], -1)

In [46]:
print( "X.shape : "+ str(X.shape) )
print( "Y.shape : "+ str(Y.shape) )

X.shape : (485, 29)
Y.shape : (485, 110)


## Splitting the inputs and outputs into a training and testing set

Then, scaling all of the inputs and outputs to (0, 1)

In [47]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2)
sss = ShuffleSplit(n_splits=1, test_size=0.2)
sss.get_n_splits(X, Y)
train_index, test_index = next(sss.split(X, Y)) 

X_train, X_test = X[train_index], X[test_index] 
Y_train, Y_test = Y[train_index], Y[test_index]
Y_std_train, Y_std_test = Y_std[train_index], Y_std[test_index]

feature_range = (-1, 1)

X_scaler = MinMaxScaler(feature_range = feature_range ).fit(X_train)
Y_scaler = MinMaxScaler(feature_range = feature_range ).fit(Y_train)
#X_scaler = StandardScaler().fit(X_train)
#Y_scaler = StandardScaler().fit(Y_train)

X_train_sc = X_scaler.transform(X_train)
X_test_sc = X_scaler.transform(X_test)

Y_train_sc = Y_scaler.transform(Y_train)
Y_test_sc = Y_scaler.transform(Y_test)

Y_std_train_sc = Y_std_train / Y_scaler.scale_

#make the torch tensors
X_train_sc = X_train_sc.astype(np.float32)
Y_train_sc = Y_train_sc.astype(np.float32)
Y_std_train_sc = Y_std_train_sc.astype(np.float32)
X_train_tensor = torch.from_numpy(X_train_sc)
Y_train_tensor = torch.from_numpy(Y_train_sc)
Y_std_train_tensor = torch.from_numpy(Y_std_train_sc)

X_test_sc = X_test_sc.astype(np.float32)
Y_test_sc = Y_test_sc.astype(np.float32)
X_test_tensor = torch.from_numpy(X_test_sc)
Y_test_tensor = torch.from_numpy(Y_test_sc)


#don't understand yet why we need to add the column for task features???
X1, X2 = X_train_tensor[:, 0].reshape(-1, 1), X_train_tensor[:, 1].reshape(-1, 1)
i1, i2 = torch.zeros(X1.shape[0], 1), torch.ones(X2.shape[0], 1)
train_X = torch.cat([torch.cat([X1, i1], -1), torch.cat([X2, i2], -1),])

for i in range(2, n_features):
    X_feat = X_train_tensor[:, i].reshape(-1, 1)
    indx   = i * torch.ones(X_feat.shape[0], 1)
    train_X = torch.cat([ train_X , torch.cat([X_feat, indx], -1),])

In [48]:
print(train_X.shape)
print(Y_std_train_tensor.shape)

torch.Size([11252, 2])
torch.Size([388, 110])


### The cell below constructs the Multitask GP model

In [49]:
num_tasks = Y.shape[1]

In [55]:
###### Using BoTorch #######
model = botorch.models.multitask.FixedNoiseMultiTaskGP(train_X, Y_train_tensor, 
                                                       Y_std_train_tensor, task_feature=-1)
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
mll = ExactMarginalLogLikelihood(model.likelihood, model)
#mll = fit_gpytorch_model(mll)
###### Using BoTorch #######

In [58]:
training_iterations = 50

# Find optimal model hyperparameters
model.train()
# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
for i in range(training_iterations):
    optimizer.zero_grad()
    output = model(train_X)
    loss = -mll(output, Y_train_tensor)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
    optimizer.step()

torch.Size([388, 110])


RuntimeError: The size of tensor a (110) must match the size of tensor b (11252) at non-singleton dimension 1

In [ ]:
# Set into eval mode
model.eval()
likelihood.eval()

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = likelihood(model(X_test_tensor))
    pred_mean = predictions.mean
    pred_lower, pred_upper = predictions.confidence_region() #returns two sigma below, above mean
    pred_std = (pred_upper - pred_lower) / 2.

In [ ]:
truth = np.linspace(-1, 1, 100)

sqrt_nplot = 4
n_plot = sqrt_nplot * sqrt_nplot
obs_indices = np.random.choice(110, n_plot, replace=False)

fig, axes = plt.subplots(sqrt_nplot, sqrt_nplot, figsize=(4*sqrt_nplot, 4*sqrt_nplot))
for i, obs_ind in enumerate(obs_indices):
    ax = axes.flatten()[i]
    ax.set_title("Obs " + str(obs_ind))
    ax.plot(truth, truth, c='r', lw=3, zorder=-1)
    ax.errorbar(Y_test_sc[:, obs_ind], pred_mean[:, obs_ind], yerr=pred_std[:, obs_ind],
                alpha=0.5, zorder=1, fmt='o')
    
    
    r2 = r2_score(Y_test_sc[:, obs_ind], pred_mean[:, obs_ind])
    ax.annotate(r'$r^2 = $' + str(round(r2, 3) ), xy = (-0.9, 0.9))
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
plt.tight_layout(True)
plt.show()

In [ ]:
sigma_res_arr = []
sigma_pred_arr = []

for iobs in range(Y.shape[1]):
    res = pred_mean[:, iobs] - Y_test_sc[:, iobs]
    res = res.numpy()
    sigma_res = np.std(res)
    sigma_pred = np.mean(pred_std[:, iobs].numpy())
    sigma_res_arr.append(sigma_res)
    sigma_pred_arr.append(sigma_pred)


plt.plot(np.arange(0, Y.shape[1]), sigma_res_arr, lw=2, label='width of residual dist.')
plt.plot(np.arange(0, Y.shape[1]), sigma_pred_arr, lw=2, color='r', label='pred. std', ls='--')
plt.legend()
plt.show()

In [ ]:
r2 = r2_score(Y_test_sc,pred_mean)
print("r2 = " + str(round(r2, 3)))